# Analyse pyBadlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **pyBadlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

import stratalAnalyse as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Loading stratigraphic file

First we need to load the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.p`X`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
# help(strata.stratalSection.__init__)

In [ ]:
folder = '/workspace/volume/GRL/rf3_sl0_ts1_dtn100/h5/'
strat = strata.stratalSection(folder)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
# help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(300) 

# Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide:
+ the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, 
+ a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and 
+ the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis
x1 = 350000.
x2 = 700000.
y1 = 350000
y2 = 350000

# Interpolation parameters
nbpts = 351
gfilt = 1

In [ ]:
# help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Stratal stacking pattern

We use **plotly** to visualise the vertival cross-section of stratal stacking apttern.

In [ ]:
#help(strata.viewSection)

In [ ]:
strata.viewSection(width = 800, height = 500, cs = strat, 
            dnlay = 2, rangeX=[220000, 350000], rangeY=[-1000,0],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

## Stratal stacking pattern coloured by depositional environments

We define paleo-depositional environments (depositional facies) based on deposition depth. For example,

- **alluvial plain**:   30m above sea level
- **shoreface**:        0 - 30m below sea level
- **shallow marine**:   30 - 100m below sea level
- **slope**:            100 - 300m below sea level
- **deep marine**:      300 - 700m below sea level
- **ocean basin**:      > 700m below sea level

To define the paleo-depositional environments, we need the paleo-depth of the sedimentary layers when they deposited. In this example, 300 outputs are recorded (**`nbout`**). We use a loop to read the output every ten timesteps (**`nstep`**). 

In addition, we also calculate the **shorleine trajectory**.

In [ ]:
nbout = 300
nstep = 10

In [ ]:
# folder = '/workspace/volume/GRL/rf3_sl0_ts1_dtn100/h5/'
strat_all = {}
strat_all[0] = strata.stratalSection(folder)
strat_all[0].loadStratigraphy(1)
k = 1
for i in range(10,nbout+1,nstep):
    strat_all[k] = strata.stratalSection(folder)
    strat_all[k].loadStratigraphy(i) 
    k += 1

## Build cross-section for each sedimentary layer

In [ ]:
npts = len(strat_all)
for i in range(0,npts):
    strat_all[i].buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

Shoreline position at a specific time is defined as the position whose elevation equals to the sea level at that time.

We first read the sea level file, and extract the value of sea-level for each stratigraphic layer.

In [ ]:
time, Sealevel = strata.readSea('/workspace/volume/GRL/data/sealevel.csv')
Time = np.array(time[::10])
sealevel = np.array(Sealevel[::10])
sealevel[0] = Sealevel[1]
timeStep = 1e5

# View sea level curve
strata.viewData(x0 = Time/timeStep, y0 = sealevel, width = 600, height = 400, linesize = 3, 
                color = '#6666FF',title='Sea-level curve',xlegend='display steps',ylegend='sea-level position')

In [ ]:
#help(strat.buildParameters)

In [ ]:
strat.buildParameters(npts,strat_all,sealevel)

## Build paleo-depositional environments structure.

In [ ]:
# Specify the range of water depth (relative to sea level, positive = below sea level) of each depositional environment
envIDs = [-20, 0, 30, 100, 300, 700]

# Get the position of depositional environments on each extracted layer
waterIDs = np.zeros((len(envIDs), npts),dtype=int)
for i in range(0,npts):
    waterIDs[:,i] = strata.depthID(cs = strat_all[i], sealevel = sealevel[i], envIDs = envIDs)
    
# Add the endpoint of deposition to the depositional environments extent
envi = np.append(waterIDs, [strat.depoend], axis=0)

In [ ]:
# Plot the stratal stacking pattern
fig = plt.figure(figsize = (7,3))
plt.rc("font", size=12)
# 
ax = fig.add_subplot(111)
layID = []
p = 0
color = ['limegreen','darkkhaki','sandybrown','khaki','c','teal']
xi00 = strat.dist/1000  # change the unit to be be km
dn=10
for i in range(0,strat.nz+1,dn):
    layID.append(i)
    if len(layID) > 1:
        for j in range(0,int(np.amax(xi00))):
            if (j<waterIDs[0][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[0])
            elif (j<waterIDs[1][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[0])
            elif (j<waterIDs[2][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[1])
            elif (j<waterIDs[3][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[2])
            elif (j<waterIDs[4][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[3])
            elif (j<waterIDs[5][i/dn]):
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[4])
            else:
                plt.fill_between([xi00[j],xi00[j+1]], [strat.secDep[layID[p-1]][j], strat.secDep[layID[p-1]][j+1]], color=color[5])
                plt.fill_between(xi00, strat.secDep[layID[p]], 0, color='white')
    p=p+1
    plt.plot(xi00,strat.secDep[i],'-',color='k',linewidth=0.3)
#     
plt.fill_between(xi00, strat.secDep[strat.nz-1], 0, color='white')
plt.plot(xi00,strat.secDep[strat.nz-1],'-',color='k',linewidth=1)

plt.plot(xi00,strat.secDep[0],'-',color='k',linewidth=1)
plt.ylabel('Elevation (m)',fontsize=12)
plt.xlabel('Distance (km)',fontsize=12)
plt.xlim( 300, 100) 
plt.ylim( -1200, 0 )
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
# 
# fig.savefig("/workspace/volume/GRL/figures/stratStacking.png", dpi=400)

# Wheeler Diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the stacking patterns of sedimentary successions and their relationship to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation. 

In [ ]:
timeMA = [x/1e6 for x in Time] # change the unit to be Myr

In [ ]:
fig = plt.figure(figsize = (7,3))
plt.rc("font", size=12)
# 
ax = fig.add_subplot(111)
colors = ['limegreen','darkkhaki','sandybrown','khaki','c','teal','white']
patch_handles = []
for i, d in enumerate(envi):
    patch_handles.append(plt.barh(timeMA,d,color=colors[i],align='edge',left=d, height=1., edgecolor = "none"))
for j in range(0,npts): 
    plt.axhline(timeMA[j], color='k', linewidth=0.3)
plt.plot(strat.shoreID, timeMA,'ko',markersize=3)
#
plt.xlim([300, 100])
plt.ylim([-0.4,31])
plt.xlabel('Distance (km)',fontsize=12)
plt.xticks(fontsize=12)
plt.ylabel('Time (Myr)',fontsize=12)
plt.yticks(fontsize=12)
# 
# fig.savefig("/workspace/volume/GRL/figures/WheelerDiagram.png", dpi=400)